# Fluent Python
> Notes By zzf 20190527

> Reference: ***Fluent Python*** *By Luciano Ramalho*

## Chapter 1  Python 数据模型
> Python 最好的品质之一是一致性。当你使用 Python 工作一会儿后，就会开始理解 Python 语言，并能正确猜测出对你来说全新的语言特征。
然而，如果你带着来自其他面向对象语言的经验进入 Python 的世界，会对 len(colleciton) 而不是 collection.len() 写法觉得不适。当你进一步理解这种不适 感背后的原因之后，会发现这个原因，和它所代表的庞大的设计思想，是形成我们通常说 的“Python 风格”（Pythonic）的关键。这种设计思想完全体现在 Python 的数据模型上，而 数据模型所描述的 API，为使用最地道的语言特性来构建你自己的对象提供了工具。

### 特殊方法 / magic method / dunder method
```py
__init__, __len__, __getitem__
```

#### Exp 1.1 纸牌

In [1]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:    
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')    
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]

##### Notes
```py
__len__(self) # 外部使用len()调用该类对象时，__len__起作用，进一层用len()调用list对象，由list对象的__len__起作用

__getitem__(self) # 外部使用[]调用该类对象时，__getitem__起作用，进一层用[]调用dict对象，由dict对象的__getitem__起作用
```

In [2]:
deck = FrenchDeck()

##### Notes
```py
# 由__len__实现
len(dect)

# 由__getitem__实现
deck[0]
deck[-1]
deck[12::13]

for card in deck: 
    print(card)

from random import choice
choice(deck) # Card(rank='3', suit='hearts')
choice(deck) # Card(rank='K', suit='spades') 
choice(deck) # Card(rank='2', suit='clubs')


```

In [3]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

##### Notes
```py
FrenchDeck.ranks # ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
FrenchDeck.ranks.index('A') # 12 -> return the index of the input element
rank_value * len(suit_values) + suit_values[card.suit] # get the 'score' of input card
```

In [4]:
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

##### Notes
```py
sorted(deck, key=spades_high) # sorted(iterable, key=function) # every element in iterable will apply the function
```

## Chapter 17  使用`future`处理并发

In [1]:
import os, time, sys, requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'

In [2]:
class Flags:
    
    @staticmethod
    def save_flag(img, filename):
        path = os.path.join(DEST_DIR, filename)
        with open(path, 'wb') as fp:
            fp.write(img)
    
    @staticmethod
    def get_flag(cc):
        url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
        resp = requests.get(url)
        return resp.content
    
    @staticmethod
    def show(text):
        print(text, end=',')
        sys.stdout.flush()
    
    @classmethod
    def download_many(cls, cc_list):
        for cc in sorted(cc_list):
            image = cls.get_flag(cc)
            cls.show(cc)
            cls.save_flag(image, cc.lower() + '.gif')

        return len(cc_list)
    
    @classmethod
    def main(cls):
        t0 = time.time()
        count = cls.download_many(POP20_CC)
        elapsed = time.time() - t0
        msg = '\n{} flags downloaded in {:.2f}s'
        print(msg.format(count, elapsed))


In [3]:
Flags.main()

BD,BR,CD,CN,DE,EG,ET,FR,ID,IN,IR,JP,MX,NG,PH,PK,RU,TR,US,VN,
20 flags downloaded in 149.46s


In [4]:
from concurrent import futures


MAX_WORKERS = 20


class Flags_Threadpool(Flags):

    @classmethod
    def download_one(cls, cc):
        image = cls.get_flag(cc)
        cls.show(cc)
        cls.save_flag(image, cc.lower() + '.gif')
        return cc

    @classmethod
    def download_many(cls, cc_list):
        workers = min(MAX_WORKERS, len(cc_list))
        with futures.ThreadPoolExecutor(workers) as executor:
            res = executor.map(cls.download_one, sorted(cc_list))

        return len(list(res))

In [5]:
Flags_Threadpool.main()

BD,ID,IN,PK,NG,IR,US,DE,PH,MX,CN,ET,CD,RU,VN,JP,BR,FR,EG,TR,
20 flags downloaded in 9.73s


In [6]:
import asyncio
import aiohttp

class Flags_Asyncio(Flags):

    @staticmethod
    async def get_flag(session, cc):
        url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
        async with session.get(url) as resp:
            return await resp.read()

    @classmethod
    async def download_one(cls, session, cc):
        image = await cls.get_flag(session, cc)
        cls.show(cc)
        cls.save_flag(image, cc.lower() + '.gif')
        return cc

    @classmethod
    async def download_many(cls, cc_list):
        async with aiohttp.ClientSession() as session:
            res = await asyncio.gather(
                *[asyncio.create_task(cls.download_one(session, cc))
                    for cc in sorted(cc_list)])

        return len(res)

In [7]:
Flags_Asyncio.main()


<coroutine object Flags_Asyncio.download_many at 0x0000028069784948> flags downloaded in 0.00s
C:\Users\Nature\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: coroutine 'Flags_Asyncio.download_many' was never awaited
  """Entry point for launching an IPython kernel.
